# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

Using Yfinance unlike most set up tutorials that use IEX Sandbox, which has been since deprecated

In [66]:
import numpy as np
import pandas as pd
import requests
import yfinance as yf
from scipy import stats
from datetime import timedelta, datetime
import math

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [20]:
df = pd.read_csv('constituents.csv')
Tickers = df['Symbol']
Tickers

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

## Defining one year return for list of tickers in this case S&P 500 stocks

We need to get one-year price returns for each stock in the S&P 500, trying to finalize the same.  

In [21]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def calculate_one_year_returns(symbols):
    """
    Calculate one-year price returns for a list of stock symbols using Yahoo Finance.
    
    Parameters:
    symbols (list): List of stock symbols to fetch data for.
    
    Returns:
    pd.DataFrame: DataFrame containing the one-year price returns for the given symbols.
    """
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)

    returns_data = {}

    for symbol in symbols:
        try:
            ticker = yf.Ticker(symbol)
            data = ticker.history(start=start_date, end=end_date)
        except:
            continue
        if not data.empty:
            start_price = data['Close'].iloc[0]
            end_price = data['Close'].iloc[-1]
            one_year_return = (end_price - start_price) / start_price
            returns_data[symbol] = one_year_return
        else:
            returns_data[symbol] = None

    returns_df = pd.DataFrame(list(returns_data.items()), columns=['Symbol', 'One_Year_Return'])
    returns_df['One_Year_Return'] = returns_df['One_Year_Return'].apply(lambda x: x * 100 if x is not None else x)  # Convert to percentage

    return returns_df
# Example usage

In [22]:
returns_df = calculate_one_year_returns(Tickers)
returns_df

$BRK.B: possibly delisted; No timezone found
$BF.B: possibly delisted; No price data found  (1d 2023-07-26 16:49:35.939782 -> 2024-07-25 16:49:35.939782)


$BF.B: possibly delisted; No price data found  (1d 2023-07-26 16:49:35.939782 -> 2024-07-25 16:49:35.939782)


,Symbol,One_Year_Return
0,MMM,15.852416
1,AOS,11.302460
2,ABT,-5.186672
3,ABBV,33.339256
4,ACN,5.984826
...,...,...
498,XYL,21.926421
499,YUM,-5.603952
500,ZBRA,5.974784
501,ZBH,-21.338492


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function to split the list into chunks of a specified size
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Example my_columns for output structure
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BK,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BX,BA,BKNG,BWA,BSX,BMY,AVGO,BR,BRO,BF.B,BLDR,BG,BXP,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,COR,CNC,CNP,CF
CHRW,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CRWD,CCI,CSX,CMI,CVS,DHR,DRI,DVA,DAY,DECK,DE,DAL,DVN,DXCM,FANG,DLR,DFS,DG,DLTR,D,DPZ,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,EG,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI
FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GE,GEHC,GEV,GEN,GNRC,GD,GIS,GM,GPC,GILD,GPN,GL,GDDY,GS,HAL,HIG,HAS,HCA,DOC,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ

In [27]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

def get_one_year_price_return(tickers: list) -> dict:
    # Define the date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    
    # Fetch the historical data for multiple tickers
    data = yf.download(tickers, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    
    # Initialize a dictionary to store returns
    returns = {}
    
    # If fetching multiple tickers, data will have a multi-level column index
    if isinstance(data.columns, pd.MultiIndex):
        for ticker in tickers:
            try:
                closing_prices = data['Close'][ticker]
                if len(closing_prices) > 0:
                    start_price = closing_prices.iloc[0]
                    end_price = closing_prices.iloc[-1]
                    one_year_return = (end_price - start_price) / start_price * 100
                    returns[ticker] = one_year_return
                else:
                    returns[ticker] = None  # Handle case where there might be no data
            except KeyError:
                returns[ticker] = None  # Handle case where ticker data is not available
    else:
        # Handle the case where data is for a single ticker
        closing_prices = data['Close']
        if len(closing_prices) > 0:
            start_price = closing_prices.iloc[0]
            end_price = closing_prices.iloc[-1]
            one_year_return = (end_price - start_price) / start_price * 100
            for ticker in tickers:
                returns[ticker] = one_year_return
        else:
            for ticker in tickers:
                returns[ticker] = None  # Handle case where there might be no data
    
    return returns

# Example usage
tickers = ["AAPL", "MSFT", "GOOGL", "TSLA", "AMZN", "META", "NFLX", "NVDA", "BABA", "V", "JPM", "DIS"]  # Add more tickers as needed
Tickers = Tickers.tolist()
returns = get_one_year_price_return(Tickers)
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
print

[**********************96%%********************  ]  482 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2023-07-26 -> 2024-07-25)


[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-07-26 -> 2024-07-25)')


The 1-year price return for MMM is 9.63%
The 1-year price return for AOS is 7.58%
The 1-year price return for ABT is -5.98%
The 1-year price return for ABBV is 24.18%
The 1-year price return for ACN is 3.85%
The 1-year price return for ADBE is 3.20%
The 1-year price return for AMD is 31.37%
The 1-year price return for AES is -20.96%
The 1-year price return for AFL is 29.37%
The 1-year price return for A is 4.43%
The 1-year price return for APD is -14.13%
The 1-year price return for ABNB is -3.96%
The 1-year price return for AKAM is 1.09%
The 1-year price return for ALB is -56.73%
The 1-year price return for ARE is -9.25%
The 1-year price return for ALGN is -27.97%
The 1-year price return for ALLE is 9.00%
The 1-year price return for LNT is 0.00%
The 1-year price return for ALL is 52.57%
The 1-year price return for GOOGL is 33.54%
The 1-year price return for GOOG is 34.48%
The 1-year price return for MO is 8.47%
The 1-year price return for AMZN is 41.11%
The 1-year price return for AMCR

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [38]:
final_df = pd.DataFrame(columns=my_columns)
rows=[]
for key, value in returns.items():
    row_df = pd.DataFrame([[key, None, value, None]], columns=my_columns)
    rows.append(row_df)
final_df = pd.concat(rows, ignore_index=True)

In [39]:
print(final_df)

    Ticker Price  One-Year Price Return Number of Shares to Buy
0      MMM  None               9.629793                    None
1      AOS  None               7.579596                    None
2      ABT  None              -5.978261                    None
3     ABBV  None              24.179009                    None
4      ACN  None               3.845055                    None
..     ...   ...                    ...                     ...
498    XYL  None              20.100153                    None
499    YUM  None              -8.266574                    None
500   ZBRA  None               5.216909                    None
501    ZBH  None             -22.049078                    None
502    ZTS  None              -6.427081                    None

[503 rows x 4 columns]


In [41]:
prices = []
for ticker in final_df['Ticker']:
    price = (yf.Ticker(ticker).info.get('previousClose'))
    prices.append(price)

In [43]:
final_df['Price'] = prices
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,103.25,9.629793,None
1,AOS,80.76,7.579596,None
2,ABT,107.26,-5.978261,None
3,ABBV,176.21,24.179009,None
4,ACN,328.68,3.845055,None
...,...,...,...,...
498,XYL,136.71,20.100153,None
499,YUM,125.95,-8.266574,None
500,ZBRA,322.09,5.216909,None
501,ZBH,110.55,-22.049078,None


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [52]:
final_df.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_df = final_df[:50]
final_df.reset_index(inplace = True)
final_df

/var/folders/g4/d2_9vd6n0cx4k51m5kmh8f2m0000gn/T/ipykernel_14249/2911281085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.sort_values('One-Year Price Return', ascending = False, inplace = True)


,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,477,VST,72.58,155.473433,None
1,1,350,NVDA,114.25,151.364078,None
2,2,430,SMCI,711.56,124.212261,None
3,3,44,ANET,325.87,98.157495,None
4,4,348,NRG,74.19,92.701305,None
5,5,222,GDDY,143.21,91.610932,None
6,6,279,KKR,115.73,90.721824,None
7,7,290,LLY,859.88,89.572083,None
8,8,412,STX,109.53,88.942550,None
9,9,191,FICO,1559.24,86.791254,None


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [67]:
def portfolio_input():
    global portfolio_size 
    portfolio_size = input("Enter your portfolio Size") 
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Not a number')
        print('enter a number')
        portfolio_size = input("Enter your portfolio Size") 
portfolio_input()
print(portfolio_size)
    

10000


In [68]:
position_size = float(portfolio_size)/50
num_shares = []
for price in final_df['Price']:
    num_shares.append(math.floor(position_size/price))

In [69]:
final_df['Number of Shares to Buy'] = num_shares
final_df

,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,477,VST,72.58,155.473433,2
1,1,350,NVDA,114.25,151.364078,1
2,2,430,SMCI,711.56,124.212261,0
3,3,44,ANET,325.87,98.157495,0
4,4,348,NRG,74.19,92.701305,2
5,5,222,GDDY,143.21,91.610932,1
6,6,279,KKR,115.73,90.721824,1
7,7,290,LLY,859.88,89.572083,0
8,8,412,STX,109.53,88.942550,1
9,9,191,FICO,1559.24,86.791254,0


## Optimizing to match real world momentum strategies 
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

`hqm` = `high-quality momentum`.

In [71]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]
hqm_df = pd.DataFrame(columns = my_columns)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: